In [131]:
from braket.circuits import Circuit, circuit
from braket.devices import LocalSimulator
from braket.aws import AwsDevice,AwsQuantumTask

import numpy as np

In [132]:
@circuit.subroutine(register=True)
def oracle():
    c = Circuit()
    c = c.cnot(0, 2)
    c = c.cnot(1, 2)
    c = c.cnot(1, 3)
    c = c.ccnot(2, 3, 4)
    c = c.cnot(0, 2)
    c = c.cnot(1, 2)
    c = c.cnot(1, 3)
    # print(c.diagram())
    return c

In [133]:
@circuit.subroutine(register=True)
def diffusor():
    c = Circuit()
    c = c.h([0, 1])
    c = c.x([0, 1])
    c = c.h([1])
    c = c.cnot(0, 1)
    c = c.h([1])
    c = c.x([0, 1])
    c = c.h([0, 1])
    # print(c.diagram())
    return c

In [134]:
c = Circuit()
c = c.h([0])
c = c.h([1])
c = c.ry(4, -1 * np.sqrt(2.0))
c = c.oracle()
c = c.diffusor()
print(c)

T  : |    0    |1|2|3|4|5|6|7|8|9|10|11|12|13|14|
                                                 
q0 : -H---------C-------C-H-X--------C--X--H-----
                |       |            |           
q1 : -H---------|-C-C---|-C-C-H-X-H--X--H--X--H--
                | | |   | | |                    
q2 : -----------X-X-|-C-X-X-|--------------------
                    | |     |                    
q3 : ---------------X-C-----X--------------------
                      |                          
q4 : -Ry(-1.41)-------X--------------------------

T  : |    0    |1|2|3|4|5|6|7|8|9|10|11|12|13|14|


In [135]:
d = LocalSimulator()
d

In [136]:
t = d.run(task_specification=c, shots=1000)
t

LocalQuantumTask('id':1cfea266-a911-4896-aeda-398cdac48d84)

In [137]:
t.result().measurement_counts

Counter({'01001': 450, '01000': 548, '11000': 1, '10000': 1})

In [138]:
sv1 = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/sv1")
am3 = AwsDevice("arn:aws:braket:us-west-1::device/qpu/rigetti/Aspen-M-3")

In [180]:
t = am3.run(c, shots=1000)
t

AwsQuantumTask('id/taskArn':'arn:aws:braket:us-west-1:602345884574:quantum-task/3fccb962-cf1d-4184-b574-51fdedf9fafc')

In [184]:
r = AwsQuantumTask(arn=t.id)
r.state()

'COMPLETED'

In [185]:
len(list(filter(lambda i: i[0] == 0 and i[1], r.result().measurements)))

237